In [1]:
import simpy
import random
from resource_monitor import Monitor
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import bokeh.plotting as bk
bk.output_notebook()

Loading BokehJS ...

In [2]:
# Generic helper class to hold information regarding to resource
# This simplifies how we pass information from main program to entity process
class Server(object):
    def __init__(self, env, name, capacity, service_rate):
        self.name = name
        self.env = env
        self.service_rate = service_rate
        self.capacity = capacity
        self.resource = simpy.Resource(env, capacity=capacity)

    def print_stats(self):
        print('\t[{}] {} using, {} in queue'.format(self.name, self.resource.count, len(self.resource.queue)))

    def get_service_time(self):
        return random.expovariate(self.service_rate)


# passenger - Entity Process
# Describe how passenger performs at the ticket office
def passenger(env, name, server, monitor):
    t_arrive = env.now
    with server.resource.request() as request:
        yield request
        t_queue = env.now - t_arrive
        # random service time based on the service rate
        service_time = server.get_service_time()
        yield env.timeout(service_time)
        t_response = env.now - t_arrive
    stats = { 't_queue': t_queue, 't_response': t_response }
    monitor.entity_logger(server.name, name, stats)


# generator - Supporting Process
# Create new passenger and then sleep for random amount of time
def passenger_generator(env, server, arrival_rate, monitor):
    i = 0
    while True:
        ename = 'Passenger#{}'.format(i)
        env.process(passenger(env, ename, server, monitor))
        next_entity_arrival = random.expovariate(arrival_rate)
        yield env.timeout(next_entity_arrival)
        i += 1


In [3]:
def model(seed=0):
    random.seed(seed)
    env = simpy.Environment()
    ticket_office = Server(env, 'office', capacity=1, service_rate=service_rate)
    monitor = Monitor()
    monitor.register('office', ticket_office.resource, ticket_office.capacity)
    env.process(passenger_generator(env, ticket_office, arrival_rate, monitor))
    env.run(until=SIMULATION_END_TIME)
    return monitor

In [4]:
def run(seed=0):
    monitor = model(seed)
    step = 20
    clocks = []
    util_stats = []
    queue_stats = []
    for i in range(step, SIMULATION_END_TIME, step):
        stats = monitor.get_stats('office', 0, i)
        clocks.append(i)
        util_stats.append(stats['stats']['util'])
        queue_stats.append(stats['stats']['queue'])
    entity = []
    tr_stats = []
    mean_tr = []
    n = 0
    mean = 0
    for e in monitor.get_resource('office')['entity']:
        t_response = e['stats']['t_response']
        tr_stats.append(t_response)
        entity.append(n)
        n += 1
        mean += t_response
        mean_tr.append(1.0*mean / n)

    r = {
        'server': {
            'util': { 'clock': clocks, 'value': util_stats },
            'queue': { 'clock': clocks, 'value': queue_stats } },
        'entity': {
            't_response': { 'id': entity, 'value': tr_stats },
            'mean_tr': { 'id': entity, 'value': mean_tr }
        }
    }
    return r

In [5]:
# plt.plot(clocks, util_stats, color="blue", linewidth=2.5, linestyle="-")
# plt.ylim(0, 1)
# plt.show()


In [6]:
MEAN_INTER_ARRIVAL_TIME = 10     # 5 time units between arrivals
MEAN_SERVICE_TIME = 4            # 8 time units for each service
SIMULATION_END_TIME = 50000

arrival_rate = 1/MEAN_INTER_ARRIVAL_TIME
service_rate = 1/MEAN_SERVICE_TIME


In [7]:
seeds = [123, 456, 789, 1111, 0]
results = []
entity_range = -1
n_runs = len(seeds)
for seed in seeds:
    r = run(seed)
    results.append(r)
    n = len(r['entity']['mean_tr']['id'])
    if entity_range == -1 or n < entity_range:
        entity_range = n

In [8]:
# mean of means
mean_of_means_x = np.arange(entity_range)
mean_of_means_y = []
for j in range(entity_range):
    mean = 0
    for r in results:
        mean += r['entity']['mean_tr']['value'][j]
    mean_of_means_y.append(1.0*mean/n_runs)

In [9]:
colors = ['green', 'red', 'blue', 'chocolate', 'teal', 'lime', 'magenta', 'dodgerblue', 'brown', 'turquoise']


In [45]:
range_x = 300
print('total = ', range_x)
# create a new plot
fig = bk.figure(
    tools="pan,box_zoom,wheel_zoom,zoom_in,zoom_out,reset,save",
    title="Time in system of {} entities".format(range_x),
    x_axis_label='Entity ID',
    y_axis_label='Time units',
    plot_width=600, plot_height=400,
)

for i in range(2):
    r = results[i]
    color = colors[i]
    entity = r['entity']['t_response']['id']
    ts_stats = r['entity']['t_response']['value']
    mean_tr = r['entity']['mean_tr']['value']
    fig.line(entity[:range_x], ts_stats[:range_x], legend_label='entity run-{}'.format(i), line_width=1, line_color=color)
    fig.line(entity[:range_x], mean_tr[:range_x], legend_label='mean of run-{}'.format(i), line_color=color, line_width=2, line_dash='dashed')


fig.line((mean_of_means_x[0], mean_of_means_x[range_x-1]), (6.667, 6.667), legend_label='theoritical', color='grey', line_width=2, line_dash='dashed')

bk.show(fig)



total =  300


In [65]:
range_x = 300
print('total = ', range_x)
# create a new plot
fig = bk.figure(
    tools="pan,box_zoom,wheel_zoom,zoom_in,zoom_out,reset,save",
    title="Mean time in system ({} entities)".format(range_x),
    x_axis_label='Entity ID',
    y_axis_label='Time units',
    plot_width=600, plot_height=400,
)

for i in range(n_runs):
    r = results[i]
    color = colors[i]
    entity = r['entity']['mean_tr']['id']
    mean_tr = r['entity']['mean_tr']['value']
    fig.line(entity[:range_x], mean_tr[:range_x], legend_label='run-{}'.format(i), line_width=1, line_color=color)

#fig.line(mean_of_means_x[:range_x], mean_of_means_y[:range_x], legend_label='mean', color='black', line_width=2)
#fig.line((mean_of_means_x[0], mean_of_means_x[range_x-1]), (6.667, 6.667), legend_label='theoritical', color='grey', line_width=2, line_dash='dashed')

bk.show(fig)

total =  300


In [28]:
print('total = ', len(mean_of_means_x))

# create a new plot
fig = bk.figure(
    tools='pan,box_zoom,wheel_zoom,zoom_in,zoom_out,reset,save',
    title='Mean time in system ({} entities)'.format(len(mean_of_means_x)),
    x_axis_label='Entity ID',
    y_axis_label='Time units',
    plot_width=600, plot_height=400,
)

n = len(results)
for i in range(n):
    r = results[i]
    color = colors[i]
    entity = r['entity']['mean_tr']['id']
    mean_tr = r['entity']['mean_tr']['value']
    fig.line(entity, mean_tr, legend_label='run-{}'.format(i), line_width=1, line_color=color)

fig.line(mean_of_means_x, mean_of_means_y, legend_label='mean of all runs', color='black', line_width=2)
fig.line((mean_of_means_x[0], mean_of_means_x[-1]), (6.667, 6.667), legend_label='theoritical', color='grey', line_width=2, line_dash='dashed')
bk.show(fig)


total =  4956


In [40]:
range_x = 300
print('total = ', range_x)
# create a new plot
fig = bk.figure(
    tools='pan,box_zoom,wheel_zoom,zoom_in,zoom_out,reset,save',
    title='Time in system ({} entities)'.format(range_x),
    x_axis_label='Entity ID',
    y_axis_label='Time units',
    plot_width=600, plot_height=400,
)

entity = results[0]['entity']['t_response']['id']
t_response = results[0]['entity']['t_response']['value']
fig.line(entity[:range_x], t_response[:range_x], legend_label='entity', line_width=1, line_color='red')
entity = results[0]['entity']['mean_tr']['id']
mean_tr = results[0]['entity']['mean_tr']['value']
fig.line(entity[:range_x], mean_tr[:range_x], legend_label='mean', color=colors[0], line_width=2)
fig.line((entity[0], entity[range_x-1]), (6.667, 6.667), legend_label='theoritical', color='grey', line_width=2, line_dash='dashed')
bk.show(fig)

total =  300


In [53]:
range_x = 50
print('total = ', range_x)
# create a new plot
fig = bk.figure(
    tools='pan,box_zoom,wheel_zoom,zoom_in,zoom_out,reset,save',
    title='Mean time in system ({} entities)'.format(len(entity)),
    x_axis_label='Entity ID',
    y_axis_label='Time units',
    plot_width=600, plot_height=400,
)

fig.line(entity[:range_x], mean_tr[:range_x], legend_label='mean(run-0)', color=colors[0], line_width=1)
fig.line(mean_of_means_x[:range_x], mean_of_means_y[:range_x], legend_label='mean of all runs', color='black', line_width=1)
fig.line((mean_of_means_x[0], mean_of_means_x[range_x-1]), (6.667, 6.667), legend_label='theoritical', color='grey', line_width=2, line_dash='dashed')

bk.show(fig)

total =  50


In [66]:
min_pct = 1
min_id = -1
interval = int(entity_range/20)
x = []
y = []
for i in range(entity_range):
    data = []
    for r in results:
        data.append(r['entity']['mean_tr']['value'][i])
    mean = np.mean(data)
    sem = st.sem(data)
    if sem != 0:
        (low, high) = st.t.interval(alpha=0.95, df=len(data)-1, loc=mean, scale=sem)
        width = high - low
    else:
        width = 0
    width_pct = 100.0 * width / mean
    x.append(i)
    y.append(width_pct)
    if i%interval == 0:
        print(i, mean, width, width_pct)
    if width_pct < min_pct:
        min_pct = width_pct
        min_id = i

print('min = ', min_id, min_pct)
print('total = ', len(x))
# create a new plot
fig = bk.figure(
    tools='pan,box_zoom,wheel_zoom,zoom_in,zoom_out,reset,save',
    title='Width %'.format(len(entity)),
    x_axis_label='Entity ID',
    y_axis_label='Percentages',
    plot_width=600, plot_height=400,
)

fig.line(x, y, legend_label='Width %', color='blue', line_width=1)
fig.line((x[0], x[-1]), (10, 10), legend_label='cut off', color='red', line_width=2, line_dash='dashed')
bk.show(fig)

0 5.077789959607184 12.881364662471999 253.68053355772315
247 6.7267504376896765 1.7347774120139334 25.789234015492156
494 6.504390805521924 0.9649929931257333 14.836024187023009
741 6.660726331649501 1.2112200772634178 18.184504466248868
988 6.592247117046523 1.3816644340081883 20.958929625611567
1235 6.722336921051133 1.1359213865439308 16.89771577777916
1482 6.821622368601332 1.4409972490454344 21.12396686861587
1729 6.721563397563704 1.3913902654456471 20.700396368350408
1976 6.700952135312873 1.033292447767149 15.420083995554517
2223 6.664547153737526 1.0073638409924275 15.115263164242009
2470 6.581718029500282 0.9196599016146845 13.97294593133018
2717 6.62466188614498 0.7406634919120751 11.18039689634759
2964 6.6312290887898815 0.7593835883831375 11.451626511695673
3211 6.601268770236591 0.7694656851752146 11.656330198893519
3458 6.627769158228392 0.8594786090000799 12.967841644469994
3705 6.626407397336672 0.8110558320854295 12.239752002139413
3952 6.676760568293926 0.7862021277

In [67]:
# create a new plot
fig = bk.figure(
    tools='pan,box_zoom,wheel_zoom,zoom_in,zoom_out,reset,save',
    title='Width %'.format(len(entity)),
    x_axis_label='Entity ID',
    y_axis_label='Percentages',
    plot_width=600, plot_height=400,
)

start_index = 4500
fig.line(x[start_index:], y[start_index:], legend_label='Width %', color='blue', line_width=1)
fig.line((x[start_index], x[-1]), (10, 10), legend_label='cut off', color='red', line_width=2, line_dash='dashed')
bk.show(fig)

In [17]:
def truncate_location(values):
    data = np.asarray(values)
    i = 0
    d_min = data.min()
    d_max = data.max()
    d = data[i]
    while d == d_min or d == d_max:
        i += 1
        d = data[i]
        d_min = data[i:].min()
        d_max = data[i:].max()
    return i

In [18]:
for r in results:
    mean_tr = r['entity']['mean_tr']['value']
    print(truncate_location(mean_tr))

print(truncate_location(mean_of_means_y))

0
2
0
3
0
0


In [64]:
mean_across_reps = np.asarray(mean_of_means_y)
mean_across = mean_across_reps.mean()
mean_of_last = []
rel_changes = []
print('mean of mean across = ', mean_across)
for i in range(0, mean_across_reps.size-1):
    v = mean_across_reps[i:]
    del_mean = v.mean()
    mean_of_last.append(del_mean)
    rel_changes.append((del_mean-mean)/mean)

# create a new plot
fig = bk.figure(
    tools='pan,box_zoom,wheel_zoom,zoom_in,zoom_out,reset,save',
    title='Mean response time across 5 replications',
    x_axis_label='',
    y_axis_label='Time units',
    plot_width=600, plot_height=400,
)
fig.line(mean_of_means_x, mean_of_means_y, legend_label='mean across reps', color='black', line_width=1)
#fig.line(mean_of_means_x[:mean_across_reps.size-1], mean_of_last, legend_label='mean of last', color='blue', line_width=1)
#fig.line((mean_of_means_x[0], mean_of_means_x[mean_across_reps.size-1]), (mean, mean), legend_label='overall mean', color='black', line_width=1, line_dash='dashed')
bk.show(fig)

# create a new plot
fig = bk.figure(
    tools='pan,box_zoom,wheel_zoom,zoom_in,zoom_out,reset,save',
    title='Relative Changes',
    x_axis_label='',
    y_axis_label='Change Ratio',
    plot_width=600, plot_height=400,
)
fig.line(mean_of_means_x[1:mean_across_reps.size-1], rel_changes, legend_label='Changes', color='blue', line_width=1)
bk.show(fig)

mean of mean across =  6.616498704588105


In [61]:
range_x = 2000
# create a new plot
fig = bk.figure(
    tools='pan,box_zoom,wheel_zoom,zoom_in,zoom_out,reset,save',
    title='Moving average',
    x_axis_label='',
    y_axis_label='Mean',
    plot_width=600, plot_height=400,
)

mean_across_reps = np.asarray(mean_of_means_y)
n = mean_across_reps.size
color_index = 0
k_ranges = [1, 50, 100, 200]
for k in k_ranges:
    print('k = ', k)
    x = np.arange(k+1, n-k, dtype=int)
    print('size = ', x.size)
    y = []
    for j in x:
        v = mean_across_reps[j-k:j+k+1].mean()
        y.append(v)
    print('---')
    fig.line(x[:range_x], y[:range_x], legend_label='k={}'.format(k), color=colors[color_index], line_width=1)
    color_index = (color_index + 1) % len(colors)

bk.show(fig)

k =  1
size =  4953
---
k =  50
size =  4855
---
k =  100
size =  4755
---
k =  200
size =  4555
---


In [21]:
mean_across_reps[2:7].size

5

In [22]:
j+k+1

4956

In [23]:
n

4956